In [ ]:
# @title 🍹smoothie
if not ('installed' in globals() and installed):
    # Clone or reset to latest remote
    !if [ -d Impractical-RIFE ]; then cd Impractical-RIFE && git fetch --all && git reset --hard origin/main && cd ..; else git clone -q https://github.com/rxys/Impractical-RIFE.git; fi

    # Install Python requirements
    !pip install -qq -r ./Impractical-RIFE/requirements.txt

    # Download weights if not already present
    !if [ ! -f RIFEv4.26_0921.zip ]; then pip install -qq gdown && gdown --id 1gViYvvQrtETBgU1w8axZSsr7YUuw31uy -O RIFEv4.26_0921.zip && unzip -o RIFEv4.26_0921.zip -d ./Impractical-RIFE/. > /dev/null; fi

    # Install yt-dlp if missing
    !if [ ! -f /usr/local/bin/yt-dlp ]; then wget -q https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp -O /usr/local/bin/yt-dlp && chmod a+rx /usr/local/bin/yt-dlp; fi
    installed = True

# Import libraries
import os
import shutil
import cv2
from google.colab import files

%cd /content/
cd = os.getcwd()


# Clean up folders from previous runs
tmp_dir = os.path.abspath("interp_videos")
downloads_dir = os.path.abspath("downloads")
final_destination = os.path.abspath("final_videos")
cookie_file = next((os.path.abspath(os.path.join(r, 'cookies.txt')) for r,_,f in os.walk('.') if 'cookies.txt' in f), None)

for d in [tmp_dir, downloads_dir]:
    if os.path.exists(d):
        shutil.rmtree(d)
    os.makedirs(d, exist_ok=True)

os.makedirs(final_destination, exist_ok=True)

# Input YouTube URL directly in the cell
# youtube_url = input("Enter YouTube URL: ")
youtube_url = "" # @param {"type": "string"}
max_res = 1440 # @param {"type":"slider","min":240,"max":3840,"step":240}

# Collect input videos into a list so we can process multiple uploads
input_videos = []
if youtube_url:
  # Run youtube-dl in shell using ! notation
  os.chdir(downloads_dir)
  cmd = f'yt-dlp --compat-options prefer-vp9-sort -f "bv+ba" --playlist-items 1 --merge-output-format mp4'
  if cookie_file:
      cmd += f' --cookies {cookie_file}'
  !{cmd} "{youtube_url}"
  video_files = [f for f in os.listdir() if os.path.isfile(f) and not f.startswith(".")]
  os.chdir(cd)
  assert len(video_files) == 1, "Unexpected number of video files downloaded."
  input_videos.append(os.path.join(downloads_dir, video_files[0]))
else:
  print("Please upload your video file(s) (any format). You can select multiple.")
  uploaded = files.upload()  # Opens file picker
  assert len(uploaded) >= 1, "Please upload at least one file."
  for filename in uploaded.keys():
      dest_path = os.path.join(downloads_dir, filename)
      shutil.move(filename, dest_path)
      input_videos.append(dest_path)

print(f"Discovered {len(input_videos)} video(s):")
for i, v in enumerate(input_videos, 1):
    print(f"  [{i}] {v}")

# Global controls
trim_duration = 0 #@param

# Helper: choose interpolation parameters

def interp_to_60(fps, target=60.0, max_multi=6, max_drop=6, skip_thresh=59.94):
    if fps >= skip_thresh:
        raise ValueError(f"Source FPS {fps} is already ≥ {skip_thresh}, no interpolation needed.")
    best = (0, 0, float('inf'))
    for m in range(2, max_multi + 1):
        for d in range(1, max_drop + 1):
            out = fps * m / d
            if out <= fps:
                continue
            err = abs(out - target)
            if err < best[2] or (err == best[2] and m < best[0]):
                best = (m, d, err)
    if best[0] == 0:
        raise RuntimeError("No valid (multi, drop) found within given bounds.")
    m, d, _ = best
    return m, d, fps * m / d

# RIFE setup
rife_dir = os.path.abspath("Impractical-RIFE")
drop_input = 1 # @param {type:"slider", min:1, max:6, step:1}

# Process each input video separately, keeping unique temp names per index
for idx, input_video in enumerate(input_videos, 1):
    base = os.path.splitext(os.path.basename(input_video))[0]
    final_output_name = os.path.join(final_destination, f"{base}_60.mp4")
    print(f"=== [{idx}] Processing: {input_video} -> {final_output_name} ===")

    # Unique temp names per video index
    renamed_video = os.path.abspath(f"temp_video_{idx}.mp4")
    trimmed_video = os.path.abspath(f"trimmed_video_{idx}.mp4")
    interp_video = os.path.join(tmp_dir, f"output_{idx}.mp4")
    combined_video = os.path.join(tmp_dir, f"combined_{idx}.mp4")

    # Rename to avoid breaking RIFE command
    os.rename(input_video, renamed_video)

    # Optional trim
    if trim_duration > 0:
      !ffmpeg -i {renamed_video} -t {trim_duration} -c copy {trimmed_video} -y
      os.remove(renamed_video)
      renamed_video = trimmed_video

    # Extract frame rate and size
    cap = cv2.VideoCapture(renamed_video)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()

    print(f"Original FPS: {fps}, dropping to {fps/drop_input if drop_input else fps}")
    fps /= drop_input if drop_input else 1

    try:
        _, _, out_fps = interp_to_60(fps)
    except ValueError as e:
        print(f"[{idx}] Skipping interpolation: {e}")
        out_fps = fps  # No interp; just pass through for mux step

    target_height = height
    min_dim = min(width, height)
    if min_dim > max_res:
        target_height = int(height * (max_res / min_dim))

    # Interpolate
    os.chdir(rife_dir)
    cmd = f'python inference_video.py --drop_input={drop_input} --fps={out_fps} --video={renamed_video} --output={interp_video}'
    if target_height < height:
      cmd += f' --fixed_height={target_height}'
    print("--Started interpolation--")
    !{cmd}
    print("--Finished interpolation--")
    os.chdir(cd)

    # Encode final video with source audio copied, cutting off audio if video is shorter
    !ffmpeg -i {interp_video} -i {renamed_video} -map 0:v -map 1:a -c:v copy -c:a copy -shortest {combined_video} -y

    shutil.move(combined_video, final_output_name)

    print(f"[{idx}] Final video saved as {final_output_name}")
    files.download(final_output_name)